In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

In [3]:
kappa = 1000*(np.pi/(180*3600*1000))*(3.08567802E13)/(3.1558149984E7)

In [4]:
def Vel_tang(pm,parallax):
    return kappa*pm/parallax

def Vel_tang_error(pm,parallax,pm_error,parallax_error):
    return np.abs(Vel_tang(pm,parallax))*(np.abs(parallax_error/parallax)+np.abs(pm/pm_error))

def U_vel(ra_sec,dec_sec,vel_ra,vel_dec,vel_rad):
    sinra = np.sin(np.deg2rad(ra_sec))
    cosra = np.cos(np.deg2rad(ra_sec))
    sindec = np.sin(np.deg2rad(dec_sec))
    cosdec = np.cos(np.deg2rad(dec_sec))
    
    return -vel_ra*sinra - vel_dec*cosra*sindec + vel_rad*cosra*cosdec

def V_vel(ra_sec,dec_sec,vel_ra,vel_dec,vel_rad):
    sinra = np.sin(np.deg2rad(ra_sec))
    cosra = np.cos(np.deg2rad(ra_sec))
    sindec = np.sin(np.deg2rad(dec_sec))
    cosdec = np.cos(np.deg2rad(dec_sec))
    
    return -vel_ra*cosra - vel_dec*sinra*sindec + vel_rad*sinra*cosdec

def W_vel(ra_sec,dec_sec,vel_ra,vel_dec,vel_rad):
    sinra = np.sin(np.deg2rad(ra_sec))
    cosra = np.cos(np.deg2rad(ra_sec))
    sindec = np.sin(np.deg2rad(dec_sec))
    cosdec = np.cos(np.deg2rad(dec_sec))
    
    return vel_dec*cosdec + vel_rad*sindec

def Vel_ra(ra_pri,dec_pri,U,V,W):
    sinra = np.sin(np.deg2rad(ra_pri))
    cosra = np.cos(np.deg2rad(ra_pri))
    
    return -U*sinra + V*cosra

def Vel_dec(ra_pri,dec_pri,U,V,W):
    sinra = np.sin(np.deg2rad(ra_pri))
    cosra = np.cos(np.deg2rad(ra_pri))
    sindec = np.sin(np.deg2rad(dec_pri))
    cosdec = np.cos(np.deg2rad(dec_pri))
    
    return -U*cosra*sindec - V*sinra*sindec + W*cosdec

def Vel_radial(ra_pri,dec_pri,U,V,W):
    sinra = np.sin(np.deg2rad(ra_pri))
    cosra = np.cos(np.deg2rad(ra_pri))
    sindec = np.sin(np.deg2rad(dec_pri))
    cosdec = np.cos(np.deg2rad(dec_pri))
    
    return U*cosra*cosdec + V*sinra*cosdec + W*sindec

### 1. Cargar la lista

In [5]:
tab = pd.read_pickle('../data_processed/je_seleccion_best.pkl')

In [6]:
tab.columns

Index(['ra1_x', 'dec1_x', 'Source_GaiaDR2_x', 'Sep_ang', 'Sep_sky',
       'radial_velocity_x', 'radial_velocity_error_x', 'parallax_x',
       'parallax_total_error_x', 'pmra_corrected_x', 'pmra_corrected_error_x',
       'pmdec_corrected_x', 'pmdec_corrected_error_x', 'phot_g_mean_mag_x',
       'phot_bp_mean_mag_x', 'phot_rp_mean_mag_x', 'RUWE_x', 'HRV_x',
       'e_HRV_x', 'D_G_x', 'Teff_x', 'Teff_error_x', 'Av_x', 'Mass_x',
       'Mass_error_x', 'U', 'U_error', 'td_x', 'td_error_x', 'tms_x',
       'tms_error_x', 'ECAC_x', 'ra1_y', 'dec1_y', 'Source_GaiaDR2_y',
       'radial_velocity_y', 'radial_velocity_error_y', 'parallax_y',
       'parallax_total_error_y', 'pmra_corrected_y', 'pmra_corrected_error_y',
       'pmdec_corrected_y', 'pmdec_corrected_error_y', 'phot_g_mean_mag_y',
       'phot_bp_mean_mag_y', 'phot_rp_mean_mag_y', 'RUWE_y', 'HRV_y',
       'e_HRV_y', 'D_G_y', 'Teff_y', 'Teff_error_y', 'Av_y', 'Mass_y',
       'Mass_error_y', 'td_y', 'td_error_y', 'tms_y', 'tms_er

### 2. Pasar la velocidad de la secundaria al marco de ref de la primaria

Crear columnas auxiliares y sus incertidumbres U,V,W

In [8]:

tab['U'] = U_vel(tab.ra1_y,tab.dec1_y,
                 Vel_tang(tab.pmra_corrected_y,tab.parallax_y),
                 Vel_tang(tab.pmdec_corrected_y,tab.parallax_y),
                 tab.radial_velocity_y)

tab['V'] = V_vel(tab.ra1_y,tab.dec1_y,
                 Vel_tang(tab.pmra_corrected_y,tab.parallax_y),
                 Vel_tang(tab.pmdec_corrected_y,tab.parallax_y),
                 tab.radial_velocity_y)

tab['W'] = W_vel(tab.ra1_y,tab.dec1_y,
                 Vel_tang(tab.pmra_corrected_y,tab.parallax_y),
                 Vel_tang(tab.pmdec_corrected_y,tab.parallax_y),
                 tab.radial_velocity_y)

tab['U_err'] = U_vel(tab.ra1_y,tab.dec1_y,
                     Vel_tang_error(tab.pmra_corrected_y,tab.parallax_y,
                                    tab.pmra_corrected_error_y,
                                    tab.parallax_total_error_y),
                     Vel_tang_error(tab.pmdec_corrected_y,tab.parallax_y,
                                    tab.pmdec_corrected_error_y,
                                    tab.parallax_total_error_y),
                     tab.radial_velocity_y)

tab['V_err'] = U_vel(tab.ra1_y,tab.dec1_y,
                     Vel_tang_error(tab.pmra_corrected_y,tab.parallax_y,
                                    tab.pmra_corrected_error_y,
                                    tab.parallax_total_error_y),
                     Vel_tang_error(tab.pmdec_corrected_y,tab.parallax_y,
                                    tab.pmdec_corrected_error_y,
                                    tab.parallax_total_error_y),
                     tab.radial_velocity_y)

tab['W_err'] = U_vel(tab.ra1_y,tab.dec1_y,
                     Vel_tang_error(tab.pmra_corrected_y,tab.parallax_y,
                                    tab.pmra_corrected_error_y,
                                    tab.parallax_total_error_y),
                     Vel_tang_error(tab.pmdec_corrected_y,tab.parallax_y,
                                    tab.pmdec_corrected_error_y,
                                    tab.parallax_total_error_y),
                     tab.radial_velocity_y)

### 3. Hacer la diferencia de velocidades y la separación entre estrellas

In [9]:
tab['vel_ra_x']  = Vel_tang(tab.pmra_corrected_x,tab.parallax_x)
tab['vel_dec_x'] = Vel_tang(tab.pmdec_corrected_x,tab.parallax_x)

In [10]:
tab['dvel_ra']  = Vel_ra(tab.ra1_x,tab.dec1_x,tab.U,tab.V,tab.W)  - tab.vel_ra_x
tab['dvel_dec'] = Vel_dec(tab.ra1_x,tab.dec1_x,tab.U,tab.V,tab.W) - tab.vel_dec_x

tab['dvel_ra_err'] = np.abs(Vel_ra(tab.ra1_x,tab.dec1_x,tab.U_err,tab.V_err,tab.W_err)) +\
                     Vel_tang_error(tab.pmra_corrected_x,tab.parallax_total_error_x,
                                    tab.pmra_corrected_error_x,tab.parallax_total_error_x)

tab['dvel_dec_err'] = np.abs(Vel_dec(tab.ra1_x,tab.dec1_x,tab.U_err,tab.V_err,tab.W_err)) +\
                      Vel_tang_error(tab.pmdec_corrected_x,tab.parallax_total_error_x,
                                     tab.pmdec_corrected_error_x,tab.parallax_total_error_x)

In [11]:
(tab.parallax_total_error_y*1000/tab.parallax_y).describe()

count    534.000000
mean      19.479006
std       12.500992
min        2.929042
25%       11.577450
50%       15.880099
75%       23.604314
max       96.013106
dtype: float64

### 4. Graficar los histogramas

In [10]:
plt.plot(tab.dvel_ra)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Extra: Obtener la relacion de masas

In [14]:
tab.plot.scatter('Mass_y','Mass_x')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='Mass_y', ylabel='Mass_x'>